**Предполагается использование Python версии 3.5 или выше**

## Опредление пола автора по тексту

На основе анализа информации из предоставленных источников стало понятно, то тексты взяты из сообщений в социальных сетях. На основании этого можно выделить следующие проблемы при анализе текстовой информации:

* Как правило, в социальных сетях пишут сообщения достаточно произвольно. Из-за могут встречаться слова с орфографическими ошибками, слова, которые не используются в обычной лексике. Например, 'лол', 'тыж' и т. д. В словах могут специально дублировать буквы. Например, 'вааау', 'круууто'. Может использоваться смесь русских  слов с иностранными: 'Cool стори'.
* Пунктуация обычно нарушается. Между предложениями может не стоять точка. Первое слово в предложении может не начинаться с большой буквы. После знаков препинания не ставится пробел. Они также могут дублироваться в текстах: '???', '!!!'.
* В текстах могут присутствовать специальные комбинации символов, которые выражают эмоции: ':)', ';(' и т. д.

Все вышеперечисленные особенности усложняют автоматическую обработку текстов. Большинство методов основываются на предположениях о соблюдении грамматических правил при написании текстов. Например, процедура морфологического анализа будет плохо работать с такими текстами.

Задачу определения пола автора только на основании текста достаточно трудно решать. Во-первых, только текстовой информации недостаточно. Даже для человека определить пол по коротким сообщениям невозможно. Например: 'Ты?', 'Xаxа', '+'. Из-за этих особенностей имеет смысл рассматривать задачу, когда число слов в сообщениях достаточно много. Можно попытаться найти какие-то психологические закономерности между мужчинами и женщинами при написании текстов.

Из-за вышеперечисленных причин было принято решение не использовать признаки на основе самих слов. Часть признаков была взята из статьи [Gender Classification of Twitter Data Based on Textual Meta-Attributes Extraction](https://www.researchgate.net/publication/293794120_Gender_Classification_of_Twitter_Data_Based_on_Textual_Meta-Attributes_Extraction)

Были выбраны следующие признаки:

* Общее длина сообщения без учёта пробелов.
* Общее число знаков пунктуации.
* Доля строчных букв в сообщении.
* Доля заглавных букв.
* Доля символов `.`, `:`, `;`, `?`, `!`, `.`, `(`, `)`.
* Общее число слов.
* Среднее число символов в слове.

Изначально было больше признаков, но от части пришлось отказаться из-за вычислительных сложностей. Использование символов `;`, `:` и т. д. обусловлено их использованием в смайликах. Учитывалось, что `(` и `)` могут выступать в качестве скобок.

Решение задачи состояло из следующих этапов:
1. Разделение всех примеров на тестовую и обучающую выборку.
2. Предвартельная обработка текста.
3. Обучение модели.
4. Проверка результата.
5. Применение модели.

## Раздленеие примеров на тестовую и обучающую

Для этого использовался файл `public.jsonlines.gz`. Так как данные умещались в оперативную память, то было принято решение загрузить всё сразу. Если для автора был известен пол, то он попадал в обучающую выборку, иначе в тестовую. Обучающая выборка также была раздедена на две для проверки работы модели.

## Предварительная обработка текста

Сначала удалялись ведущие пробелы. Так как тексты были из социальных сетей, то могли содержать полный набор символов Unicode. Необходимл было нормализовать текст с помощью модуля `unicodedata`.  Удалялись обращение к другим пользователям или сообществам: [Сергей], [I had] с помощью регулярных выражений. После производилось выделение "слов" и знаков препинания. Был выбран `TweetTokenizer` из пакета `nltk` так как он лучше подходит для текстов из социаольных сетей. Он может выделять смайлы. В конце производилась фильтрация "слов". Были взяты "слова", состоящие из знаков препинания или русских букв. Все иностранные слова не рассматривались.

## Обучение модели

Так как даннных достаточно много, то был выбран [Vowpal Wabbit](https://github.com/VowpalWabbit/vowpal_wabbit) для построениия модели. Он достаточно быстро работает и позволяет обучить моель на юбольшом объёме данных. С установкой пакета для Python 3.7 возникли проблемы. Была установлена версия, которую можно использовать из командной строки:

`vw.exe --data ./train.vw -f ./model.vw --l2 0.4 --loss_function logistic --binary --cache_file ./cache.bin  -k`

## Проверка результата 

Для проверки результатат тренировочный набор данных был разделён на две части с помощью `sklearn.model_selection.train_test_split`. 25 % данных было использовано для тестирования. Данные не помещялись в оперативную память, до было принято решение использовать только номера строк примеров в файле и значения пола. Фактически был разделён массив номеров строк. Это позволило дальше разделить все данные на две части.

## Применение модели

Vowpal Wabbit позволяет применить модель к тестовым данным с помощью указания специального аргумента в командной строке.

`vw.exe --data ./test.vw -t -f ./model.vw --binary -p ./pred.txt`